### Baseado no paper:
- Seasonality, Trend-following, and Mean reversion in Bitcoin - Matus Padysak, Radovan Vojtko

In [102]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [129]:
def strategy_backtest(ticker, start_date, end_date, initial_capital, transaction_cost, holding_period, lookback_periods):
    """
    Executa o backtest da estratégia de máximas/mínimas para um ativo específico.

    Parameters:
        ticker (str): Código do ativo a ser analisado.
        start_date (str): Data de início no formato 'YYYY-MM-DD'.
        end_date (str): Data de término no formato 'YYYY-MM-DD'.
        initial_capital (float): Capital inicial.
        transaction_cost (float): Custo operacional por operação (em decimal, ex: 0.001 para 0.1%).
        holding_period (int): Número de dias para manter a posição.
        lookback_periods (list): Lista de períodos de lookback para cálculo de máximas/mínimas.

    Returns:
        None: Exibe um gráfico interativo com a curva de capital para cada período e para Buy and Hold.
    """
    # === 1. Baixar dados do ativo ===
    data = yf.download(ticker, start=start_date, end=end_date)
    data["Return"] = data["Adj Close"].pct_change()  # Retorno diário
    data = data.dropna()

    # === 2. Inicializar gráfico ===
    fig = go.Figure()

    # === 3. Loop para cada período de lookback ===
    for period in lookback_periods:
        # Cria uma cópia do DataFrame para o período atual
        df = data.copy()

        # Calcular máximas e mínimas do período
        df[f"Max_{period}"] = df["Adj Close"].rolling(window=period).max()
        df[f"Min_{period}"] = df["Adj Close"].rolling(window=period).min()

        # Inicializar sinais de trade
        df["Signal"] = 0

        # Gerar sinais de trade
        for i in range(len(df)):
            # Sinal de compra no rompimento da máxima
            if i >= period and df["Adj Close"].iloc[i] > df[f"Max_{period}"].iloc[i - 1]:
                df.loc[df.index[i], "Signal"] = 1
            # Sinal de compra no rompimento da mínima
            elif i >= period and df["Adj Close"].iloc[i] < df[f"Min_{period}"].iloc[i - 1]:
                df.loc[df.index[i], "Signal"] = 1

        # Calcular retorno acumulado do período de holding
        df["Holding_Return"] = df["Adj Close"].pct_change(holding_period).shift(-holding_period)

        # Aplicar sinais e descontar custos operacionais
        df["Strategy_Return"] = df["Signal"] * (df["Holding_Return"] - transaction_cost)

        # Calcular curva de capital acumulada
        df["Capital"] = initial_capital * (1 + df["Strategy_Return"].fillna(0)).cumprod()

        # Adicionar curva de capital ao gráfico
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df["Capital"],
                mode="lines",
                name=f"Lookback {period}"
            )
        )

    # === 4. Calcular Buy and Hold ===
    data["Buy_Hold_Capital"] = initial_capital * (1 + data["Return"]).cumprod()

    # Adicionar curva de Buy and Hold ao gráfico
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data["Buy_Hold_Capital"],
            mode="lines",
            name="Buy and Hold"
        )
    )

    # === 5. Configurações do gráfico ===
    fig.update_layout(
        title=f"www.outspokenmarket.com - Lookback vs Buy and Hold ({ticker})",
        xaxis_title="Data",
        yaxis_title="Capital ($)",
        template="plotly_dark",
        height=700,
        width=700
    )

    # Exibir o gráfico
    fig.show()

In [136]:
# === Exemplo de uso ===
strategy_backtest(
    ticker="ETH-USD",  # Ativo
    start_date="2021-01-01",  # Data de início
    end_date="2025-12-31",  # Data de término
    initial_capital=100,  # Capital inicial
    transaction_cost=0.001,  # Custo operacional de 0.1%
    holding_period=5,  # Período de holding (5 dias)
    lookback_periods=[10, 20, 30, 40, 50]  # Períodos de lookback
    
)

[*********************100%***********************]  1 of 1 completed
